# Workflow: 

- Step 1.  For 'Position_clean' column, find all rows that contain substring combo ('intern' OR 'co-op) AND ('data science' OR 'data scientist' OR 'ML' or 'machine learning') *case insensitive*
    - replace all with 'Data Scientist Intern' 
- Step 2.  find all rows that contain substring 'data science'*case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 3. find all rows that contain substring 'analytics' *case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 4. find all rows that contain substring 'ML' or 'machine learning' (assume: exclude 'AI' roles) *case-insensitive*
    - replace case by case with 'xx Data Scientist' 
 
- Step 5. Other special cases (from examining df.Position. unique()): replace case by case with 'xx Data Scientist' 
  
- Step 6: find all rows that do NOT contain substring *case-insensitive* 'Data Scientist'(e.g. 'software engineer', 'AI research scientist', 'data engineer', 'software developer', 'database developer', 'data analyst') -> replace with np.nan (for easier removal of these rows later) 


[see here for str.replace](https://stackoverflow.com/questions/39768547/replace-whole-string-if-it-contains-substring-in-pandas)

**note: order matters! e.g. '*Senior* *director*, data scientist' - need to change to LDS, before SR**

- Step 7A. replace any substrings 'lead' or 'director' or 'manager' or 'principal' -> 'Lead Data Scientist'
- Step 7B. replace any substrings 'sr' or 'senior' -> 'Senior Data Scientist'
- Step 7C. replace any substrings jr' or 'junior' -> 'Junior Data Scientist'
- Step 7D. replace substrings NOT 'intern' NOT 'junior' NOT 'senior' NOT 'lead' NOT NaN  -> 'Data Scientist' 


AFTER merging with Sasha's cleaned columns, do following: 
- drop rows that have NaN in the 'Position_clean' column [see how here](https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan)
- check and drop duplicates
- drop other rows with certain NaN fields

In [1]:
import pandas as pd
import numpy as np

In [2]:
INTERN = 'Data Scientist Intern'
JR = 'Junior Data Scientist'
SR = 'Senior Data Scientist'
DS = 'Data Scientist'
LDS = 'Lead Data Scientist'
rm = np.nan

In [3]:
df_uk = pd.read_csv('jack_uk.csv', index_col = 0)

In [4]:
df_uk.shape

(1048, 12)

In [5]:
# duplicate
df_uk['Position_clean'] = df_uk.loc[:, 'Position']

In [6]:
df_uk.head()

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
0,OpenMarket,We’ve got a pretty good thing going on here at...,NaN,Enterprise Software & Network Solutions,"London, England",Principal Data Scientist,4.5,$100 to $500 million (USD),Information Technology,201 to 500 Employees,Subsidiary or Business Segment,1999.0,Principal Data Scientist
1,causaLens,Data Scientist - Applied Science\r\r\r\r\r\r\r...,NaN,Enterprise Software & Network Solutions,"London, England",Data Scientist - Applied Science,4.5,Unknown / Non-Applicable,Information Technology,1 to 50 Employees,Company - Private,2017.0,Data Scientist - Applied Science
2,Northrop Grumman UK,Are you an experienced Data Scientist looking ...,NaN,Aerospace & Defense,"Cheltenham, England",Principal Data Scientist,3.8,$10+ billion (USD),Aerospace & Defense,10000+ Employees,Company - Public,1939.0,Principal Data Scientist
3,LGC Limited,We are looking for an Immunogenicity Regulated...,NaN,NaN,"Fordham, Cambridgeshire, East of England, England",Senior Scientist/Regulated Project Manager - I...,NaN,Less than $1 million (USD),NaN,Unknown,Company - Public,NaN,Senior Scientist/Regulated Project Manager - I...
4,Neilson Financial Services,Data Scientist - Decision Science\r\r\r\r\r\r\...,NaN,Insurance Agencies & Brokerages,"Slough, Berkshire, South East England, England",Data Scientist,3.1,$10 to $25 million (USD),Insurance,51 to 200 Employees,Company - Private,2012.0,Data Scientist


In [7]:
df_uk['Position'].unique()

array(['Principal Data Scientist', 'Data Scientist - Applied Science',
       'Senior Scientist/Regulated Project Manager - Immunogenicity',
       'Data Scientist',
       'Research Scientist Peri-and-Post Approval Studies (PPAS)',
       'Junior Data Scientist', 'Research Scientist Evidence Synthesis',
       'Senior Data Scientist',
       'Research Associate III/Senior Research Associate Data Collection Peri- and Post Approval Studies',
       'Junior Data Scientist - London',
       'Data Scientist - Engineering and Product',
       'Data Scientist - Reinforcement Learning',
       'Experienced Physical Sciences Scientist',
       'Research Data Scientist, Intern - Infrastructure Strategy',
       'Lead Scientist - Next Generation Sequencing',
       'Junior Data Scientist - Home based', 'Data Scientist Intern',
       'Data Science - Deep Learning Research',
       'Artificial Intelligence – Data Scientist',
       'Data Scientist (Menu Tribe)', 'Lead Data Scientist',
       'Dat

In [8]:
# Step 1. DS intern 
df_uk.loc[df_uk['Position'] == 'Internal only -Head Data Scientist, HateLab, School of Social Sciences', 'Position_clean'] = LDS
df_uk.loc[df_uk['Position'] == 'International Data Scientist', 'Position_clean'] = DS
# placement|intern|co-op
# BMW Plant Hams Hall, Birmingham - Quality Data Scientist - 13 Month Placement
# GE Aviation Data Science - 12 Month Placement
# GE Aviation Data Science - 12 Month Placement

In [9]:
s = ['intern', 'co-op', 'co op', 'placem']
cond_or1 = '|'.join(s)
t = ["data scien", "machine learn"]
cond_or2 = '|'.join(t)
bool_intern = (df_uk['Position_clean'].str.contains(cond_or1, case = False))& \
               ( (df_uk['Position_clean'].str.contains(cond_or2, case = False)) |\
                (df_uk['Position_clean'].str.contains('ML', case = True))
               ) 
df_uk[bool_intern] # display rows

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
19,Facebook,Facebook's mission is to give people the power...,NaN,Internet,"London, England","Research Data Scientist, Intern - Infrastructu...",4.3,$5 to $10 billion (USD),Information Technology,10000+ Employees,Company - Public,2004.0,"Research Data Scientist, Intern - Infrastructu..."
24,RELX,Undergraduate Data Scientists Industrial Place...,NaN,Enterprise Software & Network Solutions,"London, England",Data Scientist Intern,3.7,$10+ billion (USD),Information Technology,10000+ Employees,Company - Public,1880.0,Data Scientist Intern
32,LexisNexis Risk Solutions,Undergraduate Data Scientists Industrial Place...,NaN,Consulting,"London, England",Data Scientist Intern,3.8,$2 to $5 billion (USD),Business Services,1001 to 5000 Employees,Company - Public,2000.0,Data Scientist Intern
501,BMW Group UK,We believe in creating an environment where ou...,NaN,Transportation Equipment Manufacturing,"Warwick, West Midlands, England","BMW Plant Hams Hall, Birmingham - Quality Data...",4.1,Unknown / Non-Applicable,Manufacturing,5001 to 10000 Employees,Subsidiary or Business Segment,NaN,"BMW Plant Hams Hall, Birmingham - Quality Data..."
799,BMW Group UK,We believe in creating an environment where ou...,NaN,Transportation Equipment Manufacturing,"Warwick, West Midlands, England","BMW Plant Hams Hall, Birmingham - Quality Data...",4.1,Unknown / Non-Applicable,Manufacturing,5001 to 10000 Employees,Subsidiary or Business Segment,NaN,"BMW Plant Hams Hall, Birmingham - Quality Data..."
834,General Electric,Job Description Summary\r\r\r\r\r\r\r\r\r\nIn ...,NaN,Industrial Manufacturing,"Eastleigh, England","GE Aviation Data Science - 12 Month Placement,...",3.9,$10+ billion (USD),Manufacturing,10000+ Employees,Company - Public,1892.0,"GE Aviation Data Science - 12 Month Placement,..."
835,GE Aviation,Job Description Summary\r\r\r\r\r\r\r\r\r\nIn ...,NaN,Aerospace & Defense,"Eastleigh, England",GE Aviation Data Science - 12 Month Placement,3.7,$500 million to $1 billion (USD),Aerospace & Defense,1001 to 5000 Employees,Subsidiary or Business Segment,1917.0,GE Aviation Data Science - 12 Month Placement


In [10]:
df_uk.loc[bool_intern, 'Position_clean'] = INTERN
# check
df_uk[bool_intern][['Position', 'Position_clean']]

,Position,Position_clean
19,"Research Data Scientist, Intern - Infrastructu...",Data Scientist Intern
24,Data Scientist Intern,Data Scientist Intern
32,Data Scientist Intern,Data Scientist Intern
501,"BMW Plant Hams Hall, Birmingham - Quality Data...",Data Scientist Intern
799,"BMW Plant Hams Hall, Birmingham - Quality Data...",Data Scientist Intern
834,"GE Aviation Data Science - 12 Month Placement,...",Data Scientist Intern
835,GE Aviation Data Science - 12 Month Placement,Data Scientist Intern


In [11]:
## Step 2
bool_2 = df_uk['Position'].str.contains('data science', case = False)
df_uk[bool_2]['Position'].unique() # display

array(['Data Science - Deep Learning Research',
       'Data Science - Machine Learning Research',
       'Data Science Engineer', 'Data Science Manager',
       'Head of Life Data Science', 'Data Science Lead, Financial Crime',
       'Data Science Lead, Reliability Analytics', 'Data Science',
       'Senior Strategic Consultant - Data Science & Analytics',
       'Senior Data Science Manager',
       'Senior Data Science Manager - Strategic Analytics',
       'Data Science Campus - Programme Manager',
       'GE Aviation Data Science - 12 Month Placement, Eastleigh',
       'GE Aviation Data Science - 12 Month Placement', 'VP Data Science',
       'VP of Data Science', 'Senior Software Engineer - Data Science',
       'Head of Data Science', 'Head of Data Science Engineering',
       'Lead Data Scientist / Data Science Director',
       'Sr. Manager, Data Science',
       'Director of Data Products and Data Science',
       'Director of Decision & Data Science', 'Data Science Directo

In [12]:
# remove SE eng
df_uk.loc[df_uk['Position'] == 'Senior Software Engineer - Data Science', 'Position_clean'] = rm

In [13]:
# Data Science
df_uk.loc[df_uk['Position'] == 'Data Science', 'Position_clean'] = DS

In [14]:
# lead ds
bool_lds = (df_uk['Position_clean'].str.contains('VP|head|manager|direct|lead|principal', case = False)) & \
           (df_uk['Position_clean'].str.contains('data science', case = False))
df_uk.loc[bool_lds, 'Position_clean'] = LDS
# sr ds
bool_sr = (df_uk['Position_clean'].str.contains('senior|sr', case = False)) & \
           (df_uk['Position_clean'].str.contains('data science', case = False))
df_uk.loc[bool_sr, 'Position_clean'] = SR
# ds
bool_ds = (df_uk['Position_clean'].str.contains('engineer|learning res', case = False)) & \
           (df_uk['Position_clean'].str.contains('data science', case = False))
df_uk.loc[bool_ds, 'Position_clean'] = DS

In [15]:
# chceck
df_uk[bool_2][['Position','Position_clean']]

,Position,Position_clean
25,Data Science - Deep Learning Research,Data Scientist
33,Data Science - Machine Learning Research,Data Scientist
86,Data Science Engineer,Data Scientist
115,Data Science Manager,Lead Data Scientist
127,Data Science Manager,Lead Data Scientist
135,Head of Life Data Science,Lead Data Scientist
152,Data Science Manager,Lead Data Scientist
331,Data Science Manager,Lead Data Scientist
352,"Data Science Lead, Financial Crime",Lead Data Scientist
382,"Data Science Lead, Reliability Analytics",Lead Data Scientist


In [16]:
## step 3
bool_3 = df_uk['Position'].str.contains('analytics', case = False)
df_uk[bool_3]['Position'].unique()

array(['Data & Analytics Consultant',
       'Data & Analytics Consultant (m/f/d)',
       'Research Scientist – Analytics Group',
       'Senior Product Manager (Data Visualisation / Analytics)',
       'Senior Data Scientist- (Machine Learning & Advanced Analytics)',
       'Senior Analyst - Strategy & Analytics',
       'Customer Analytics Data Scientist - Senior Associate',
       'Senior Data Scientist, Product Analytics (Belfast, UK)',
       'Head of Data Analytics',
       'Senior Data Scientist, Advanced Analytics',
       'Data Science Lead, Reliability Analytics',
       'Senior Strategic Consultant - Data Science & Analytics',
       'Senior Data Science Manager - Strategic Analytics',
       'Data Scientist – Customer Analytics',
       'Senior Data Analytics Consultant'], dtype=object)

In [17]:
# remove PM
df_uk.loc[df_uk['Position'] == 'Senior Product Manager (Data Visualisation / Analytics)', 'Position_clean'] = rm

In [18]:
# lead ds
bool_lds = (df_uk['Position_clean'].str.contains('VP|head|manag|director|lead|principal', case = False, na = False)) & \
           (df_uk['Position_clean'].str.contains('analytics', case = False, na = False))
df_uk.loc[bool_lds, 'Position_clean'] = LDS
# sr ds
bool_sr = (df_uk['Position_clean'].str.contains('senior|sr', case = False, na = False)) & \
           (df_uk['Position_clean'].str.contains('analytics', case = False, na = False))
df_uk.loc[bool_sr, 'Position_clean'] = SR
# ds
bool_ds = df_uk['Position_clean'].str.contains('analytics', case = False, na = False)
df_uk.loc[bool_ds, 'Position_clean'] = DS

In [19]:
# check 
df_uk[bool_3][['Position', 'Position_clean']]

,Position,Position_clean
48,Data & Analytics Consultant,Data Scientist
98,Data & Analytics Consultant (m/f/d),Data Scientist
125,Research Scientist – Analytics Group,Data Scientist
154,Senior Product Manager (Data Visualisation / A...,NaN
227,Senior Data Scientist- (Machine Learning & Adv...,Senior Data Scientist
234,Senior Analyst - Strategy & Analytics,Senior Data Scientist
335,Customer Analytics Data Scientist - Senior Ass...,Senior Data Scientist
337,"Senior Data Scientist, Product Analytics (Belf...",Senior Data Scientist
344,Head of Data Analytics,Lead Data Scientist
363,"Senior Data Scientist, Product Analytics (Belf...",Senior Data Scientist


In [20]:
## Step 4 - no need 
bool_ml =  df_uk['Position'].str.contains('data', case = False, na = False)&\
            (df_uk['Position'].str.contains('machine learn', case = False, na = False)|\
         df_uk['Position'].str.contains('ML', case = True, na = False))
df_uk[bool_ml][['Position', 'Position_clean']]

,Position,Position_clean
33,Data Science - Machine Learning Research,Data Scientist
64,Senior Data Scientist/ML Engineer,Senior Data Scientist/ML Engineer
130,Senior Data Scientist - Machine Learning,Senior Data Scientist - Machine Learning
182,Sr. Data Scientist - ML Labs,Sr. Data Scientist - ML Labs
199,Sr. Data Scientist - ML Labs,Sr. Data Scientist - ML Labs
227,Senior Data Scientist- (Machine Learning & Adv...,Senior Data Scientist
244,Senior Data Scientist/ML Engineer,Senior Data Scientist/ML Engineer
370,Senior Data Scientist/ Machine Learning Developer,Senior Data Scientist/ Machine Learning Developer
372,Senior Data Scientist / ML Engineer,Senior Data Scientist / ML Engineer
377,Senior Data Scientist - Machine Learning,Senior Data Scientist - Machine Learning


In [21]:
## Step 5 
df_uk.loc[df_uk['Position'] == 'Senior Data & Applied Scientist', 'Position_clean'] = SR
df_uk.loc[df_uk['Position'] == 'Senior/Principal Statistician, Statistical Innovation', 'Position_clean'] = SR

In [22]:
bool_dec1 = df_uk['Position_clean'].str.contains('decision scien', case = False, na = False ) &\
            df_uk['Position_clean'].str.contains('principal|lead|head|manage|direct|VP', case = False, na = False )
bool_dec2 = df_uk['Position_clean'].str.contains('decision scien', case = False, na = False ) &\
            df_uk['Position_clean'].str.contains('senior|sr', case = False, na = False )

df_uk.loc[bool_dec1, 'Position_clean'] = LDS
df_uk.loc[bool_dec2, 'Position_clean'] = SR

In [23]:
bool_spec1 = df_uk['Position_clean'].str.contains('head of data|data insight consultant', case = False, na = False )
bool_spec2 = df_uk['Position_clean'].str.contains('data driven|Data Analyst/Sci', case = False, na = False )

df_uk.loc[bool_spec1, 'Position_clean'] = LDS
df_uk.loc[bool_spec2, 'Position_clean'] = SR

In [24]:
## Step 6 
bool_notDS = df_uk['Position_clean'].str.contains('Data Scientist', case = False) == False
#df_uk[bool_notDS]['Position_clean'].unique()
df_uk.loc[bool_notDS, 'Position_clean'] = rm

In [25]:
# check no. of unique data scientist listings left after removing irrelevant rows
df_uk[df_uk['Position_clean'].str.contains('Data Scientist', case = False) == True]['Position'].unique().shape

(133,)

In [26]:
df_uk['Position'].unique().shape

(336,)

## Out of 336 *unique* UK listings, get 133 *unique* data scientist (includes: 5 intern,  3 Jr.,   42 Sr.,  40 Lead,   43 data scientist) listings

In [27]:
## Step 7 
bool_lds = df_uk['Position_clean'].str.contains('lead|principal|manage|direct|VP|head', case = False, na = False)
#df_ca[bool_lds]['Position'].unique().shape 
#df_uk[bool_lds]['Position'].unique()
df_uk.loc[bool_lds, 'Position_clean'] = LDS

In [28]:
bool_sr = df_uk['Position_clean'].str.contains('senior|sr', case = False, na = False)
#df_uk[bool_sr]['Position'].unique().shape
df_uk.loc[bool_sr, 'Position_clean'] = SR

In [33]:
bool_jr = df_uk['Position_clean'].str.contains('junior|jr|new grad|entry', case = False, na = False)
df_uk[bool_jr]['Position'].unique().shape 
df_uk.loc[bool_jr, 'Position_clean'] = JR

In [35]:
bool_remain = (df_uk['Position_clean'].str.contains('senior', case = False, na = False) == False) &\
                (df_uk['Position_clean'].str.contains('junior', case = False, na = False) == False) &\
                (df_uk['Position_clean'].str.contains('lead', case = False, na = False) == False) &\
                (df_uk['Position_clean'].str.contains('intern', case = False, na = False) == False) &\
                df_uk['Position_clean'].notna()
df_uk.loc[bool_remain, 'Position_clean'] = DS

In [37]:
# check 
df_uk['Position_clean'].unique()

array(['Lead Data Scientist', 'Data Scientist', nan,
       'Junior Data Scientist', 'Senior Data Scientist',
       'Data Scientist Intern'], dtype=object)

In [38]:
df_uk[['Position', 'Position_clean']]

,Position,Position_clean
0,Principal Data Scientist,Lead Data Scientist
1,Data Scientist - Applied Science,Data Scientist
2,Principal Data Scientist,Lead Data Scientist
3,Senior Scientist/Regulated Project Manager - I...,NaN
4,Data Scientist,Data Scientist
5,Research Scientist Peri-and-Post Approval Stud...,NaN
6,Junior Data Scientist,Junior Data Scientist
7,Junior Data Scientist,Junior Data Scientist
8,Junior Data Scientist,Junior Data Scientist
9,Junior Data Scientist,Junior Data Scientist


In [39]:
# write to csv
df_uk.to_csv('elisa_position_uk.csv')

In [40]:
# read in to check 
df = pd.read_csv('elisa_position_uk.csv', index_col = 0)
df.shape

(1048, 13)

In [45]:
df_uk[df_uk['Position'] == 'Data Scientist - Managed Rules']

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
450,Cloudflare,About Us\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r...,NaN,Internet,"London, England",Data Scientist - Managed Rules,4.0,Unknown / Non-Applicable,Information Technology,1001 to 5000 Employees,Company - Public,2010.0,Lead Data Scientist
751,Cloudflare,About Us\r\r\r\r\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r...,NaN,Internet,"London, England",Data Scientist - Managed Rules,4.0,Unknown / Non-Applicable,Information Technology,1001 to 5000 Employees,Company - Public,2010.0,Lead Data Scientist
